In [1]:
import pandas as pd
import sqlite3
from tqdm import tqdm_notebook as tqdm

In [2]:
#pd.__version__

In [3]:
pd.options.display.max_columns = None #display all columns in DF
pd.options.display.max_rows = None #display all rows in DF

## 1-Check if the data chunks look normal:

In [ ]:
db = sqlite3.connect('../data/nppes_lite.sqlite')

query = '''
SELECT *
FROM zip_cbsa
LIMIT 5
'''

df = pd.read_sql(query,db)
df = pd.DataFrame(df)

db.close()

df.head(5)

## 2- Filter columns of interest, taxonomy code and entity_type_code:

* For simplicity (since we only have 2 weeks), for_npi code = 1 (provider) and to_npi code = 2 (facility). <br>
* We need the provider's taxonomy code, which is contained in one of the 'Healthcare Provider Taxonomy Code' columns. A provider can have up to 15 taxonomy codes, but we want the one which has Primary Switch = Y in the associated 'Healthcare Provider Primary Taxonomy Switch field.

In [5]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

In [6]:
df = pd.DataFrame()

def run_query(n):
    query = '''
    SELECT npi, 
    entity_type_code,
    [provider_organization_name_(legal_business_name)],
    [provider_last_name_(legal_name)],
    provider_first_name,
    provider_middle_name,
    provider_name_prefix_text,
    provider_name_suffix_text,
    provider_credential_text,
    provider_first_line_business_practice_location_address,
    provider_second_line_business_practice_location_address,
    provider_business_practice_location_address_city_name,
    provider_business_practice_location_address_state_name,
    provider_business_practice_location_address_postal_code,
    healthcare_provider_taxonomy_code_{0} AS healthcare_provider_taxonomy_code
    FROM nppes_raw
    WHERE healthcare_provider_primary_taxonomy_switch_{0} = 'Y' AND
    entity_type_code IN (1,2) AND
    provider_business_practice_location_address_state_name = 'TN'
    '''.format(n)
    
    df = pd.read_sql(query, db)
    return df 

for i in range (1,16):
    results = run_query(i)
    df = df.append(results)

In [7]:
### To put the clean version of nppes file in the master database
df.to_sql('nppes', db, if_exists = 'append', index = False) 

In [8]:
db.close() 

## 3- Rename columns:

In [ ]:
###SQLite needs to be 3.25 or later
sqlite3.sqlite_version

In [10]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#rename table columns
db.execute('ALTER TABLE nppes RENAME COLUMN [provider_organization_name_(legal_business_name)] TO org_name')
db.execute('ALTER TABLE nppes RENAME COLUMN [provider_last_name_(legal_name)] TO last_name')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_first_name TO first_name')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_middle_name TO middle_name')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_name_prefix_text TO name_prefix')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_name_suffix_text TO name_suffix')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_credential_text TO provider_credential')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_first_line_business_practice_location_address TO address_1')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_second_line_business_practice_location_address TO address_2')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_business_practice_location_address_city_name TO city')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_business_practice_location_address_state_name TO state')
db.execute('ALTER TABLE nppes RENAME COLUMN provider_business_practice_location_address_postal_code TO zip')
db.execute('ALTER TABLE nppes RENAME COLUMN healthcare_provider_taxonomy_code TO taxonomy_code')

### Test the query
query = """
SELECT * 
FROM nppes
LIMIT 25
"""
test = pd.read_sql(query,db)

db.close() 

In [11]:
### Check columns names
test.columns.values.tolist()

['npi',
 'entity_type_code',
 'org_name',
 'last_name',
 'first_name',
 'middle_name',
 'name_prefix',
 'name_suffix',
 'provider_credential',
 'address_1',
 'address_2',
 'city',
 'state',
 'zip',
 'taxonomy_code']

In [12]:
test.head(10)

,npi,entity_type_code,org_name,last_name,first_name,middle_name,name_prefix,name_suffix,provider_credential,address_1,address_2,city,state,zip,taxonomy_code
0,1841293891,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,208600000X
1,1659374601,1.0,None,OBERDICK,WENDY,TIPTON,None,None,MD,105 W STONE DR,STE 1F,KINGSPORT,TN,376603365.0,207Q00000X
2,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,372031632.0,363L00000X
3,1003819046,1.0,None,NYLANDER,BARBARA,H,None,None,M.D.,345 23RD AVE N,SUITE 209,NASHVILLE,TN,372031513.0,207VG0400X
4,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,370551303.0,207Q00000X
5,1396748323,1.0,None,CAMPBELL,LAURA,IVEY,DR.,None,PHARMD,272 HIGHWAY 11 E,None,BULLS GAP,TN,37711.0,183500000X
6,1922001957,1.0,None,PRESLEY,RICHARD,E,None,None,M.D.,2011 MURPHY AVE,STE 302,NASHVILLE,TN,372032023.0,207V00000X
7,1104829134,1.0,None,FALCONER,RANDALL,None,DR.,None,MD,1728 N EASTMAN RD,None,KINGSPORT,TN,376642371.0,207Y00000X
8,1073516001,1.0,None,ROSS,DAVID,L,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,370552850.0,207R00000X
9,1144223298,1.0,None,SMITH,KIRBY,L,DR.,None,MD,80 HUMPHREYS CENTER DR STE 330,None,MEMPHIS,TN,381202363.0,207RH0003X
